In [1]:
from ast import If
import sys
import cv2
import numpy as np

src1 = cv2.imread('kids1.png')
src2 = cv2.imread('kids2.png')
mask = cv2.imread('kids1_mask.bmp', cv2.IMREAD_GRAYSCALE)
if src1 is None or src2 is None or mask is None :
    print('Image load failed!')
    sys.exit()



이미지 로드후 관심영역 지정

In [2]:
##x,y,w,h = cv2.selectROI(src1)



src1_ycrcb = cv2.cvtColor(src1, cv2.COLOR_BGR2YCrCb)
src2_ycrcb = cv2.cvtColor(src2, cv2.COLOR_BGR2YCrCb)

##crop = src1_ycrcb[y:y+h, x:x+w]

channels = [1, 2] ## Y채널 안쓰고 색상정보만 볼거라 0 제외

one = np.mat(1.0)

cr_bins = 256
cb_bins = 256 ## 정규화 분해능?

histsize = [cr_bins, cb_bins]
cr_range = [0, 256]
cb_range = [0, 256]

ranges = cr_range + cb_range

hist = cv2.calcHist([src1_ycrcb], channels, mask, histsize, ranges)

#마스킹 된 부분에서의 히스토그램 계산
hist_norm = cv2.normalize((cv2.log(hist + one)), None, 0, 255,
                          cv2.NORM_MINMAX, cv2.CV_8U)


#log scale로 hist 사용하는것은 히스토그램이 큰값이 있으면 작은것들이 반영되기 힘들기 때문에 log 사용
##이미지 전체에 대한 히스토그램을 구하려면 None, 특정 구역을 정하고 싶으면 mask 파일 사용하면됨
###이미지 넘겨줄때 [이미지] 이 형식으로 전달해야함

 

In [3]:
backproject = cv2.calcBackProject([src2_ycrcb], channels, hist, ranges, 1)
#src1의 mask 부분에서의 히스토그램 값을 가지는 것들과 동일한 부분을 backproject

dst = cv2.copyTo(src2, backproject)

cv2.imshow('backproj', backproject)
cv2.imshow('hist_norm', hist_norm)
cv2.imshow('dst', dst)

cv2.waitKey()
cv2.destroyAllWindows()